# Modelos de Regresión Logística
Repo: https://github.com/Diegoval-Dev/DM-HDT7

- Gerson Ramirez - 22281
- Diego Valenzuela - 22309

## Paso 1: Crear variables dicotómicas para las categorías de precio

Para aplicar modelos de clasificación binaria como la regresión logística, es necesario transformar la variable categórica que representa el precio de la vivienda en variables dicotómicas. En ejercicios anteriores se clasificaron las viviendas en tres categorías: `cara`, `media` y `económica`.

En este paso, crearemos tres nuevas columnas binarias:
- `cara_o_no`: toma valor 1 si la vivienda es cara, 0 en caso contrario.
- `media_o_no`: toma valor 1 si la vivienda es de precio medio, 0 en caso contrario.
- `economica_o_no`: toma valor 1 si la vivienda es económica, 0 en caso contrario.

Estas variables permitirán construir modelos independientes para predecir cada categoría.


In [4]:
import pandas as pd

df = pd.read_csv("processed_data.csv")

print("Categorías únicas:", df['PriceCategory'].unique())

df['cara_o_no'] = (df['PriceCategory'] == 'Cara').astype(int)
df['media_o_no'] = (df['PriceCategory'] == 'Intermedia').astype(int)
df['economica_o_no'] = (df['PriceCategory'] == 'Economica').astype(int)

df[['PriceCategory', 'cara_o_no', 'media_o_no', 'economica_o_no']].head()



Categorías únicas: ['Cara' 'Intermedia' 'Economica']


,PriceCategory,cara_o_no,media_o_no,economica_o_no
0,Cara,1,0,0
1,Intermedia,0,1,0
2,Cara,1,0,0
3,Intermedia,0,1,0
4,Cara,1,0,0


### Análisis del Paso 1: Variables dicotómicas

Las nuevas columnas `cara_o_no`, `media_o_no` y `economica_o_no` fueron creadas exitosamente a partir de la variable categórica `PriceCategory`. Cada una representa una clase como una variable binaria (1 si pertenece a esa clase, 0 si no).

Este paso es esencial para aplicar modelos de clasificación binaria como la regresión logística. Al usar estas variables, se podrá entrenar un modelo independiente para predecir si una vivienda pertenece o no a cada una de las categorías definidas.

El resultado obtenido muestra que las variables se asignaron correctamente. Por ejemplo, una vivienda con `PriceCategory = "Cara"` tiene `cara_o_no = 1` y `media_o_no` y `economica_o_no` iguales a 0.
